<a href="https://colab.research.google.com/github/reallygooday/60daysofudacity/blob/master/Hemingway.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://colab.research.google.com/drive/12XhM8q_QG5wIZTfOFAdlD4GsXeMUPrhB#scrollTo=2_5-XQJw-GC8

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
!ls '/content/drive/My Drive/Text Generator'

'generated_script2019-07-30 17:15:13.735175.txt'
 oliver.txt
 place2.txt
 place.txt
'preprocessed2019-07-30 17:06:00.732826.p'
'preprocessed2019-07-30 17:06:42.389736.p'
'preprocessed2019-07-30 17:14:12.698497.p'
'trained_scriptGenModel2019-07-30 17:15:09.497845.pt'
'trained_scriptGenModel2019-07-30 17:16:10.999422.pt'
'trained_scriptGenModel2019-07-30 17:16:37.995327.pt'


In [0]:
with open('/content/drive/My Drive/Text Generator/place.txt', 'r') as f:
  transcript = f.read()

In [0]:
lines = transcript.split('\n')

In [0]:
line_lengths = [len(line.split()) for line in lines]

In [0]:
def getUniqueWords(text):
  words = text.lower().split()
  return list(set(words))

In [8]:
import numpy as np

print('number of lines: ', len(lines))
print('average number of words in a sentence: ', round(np.average(line_lengths), 1))
print('number of words: ', len(transcript.lower().split()))
print('number of unique words: ', len(getUniqueWords(transcript)))

number of lines:  406
average number of words in a sentence:  3.5
number of words:  1441
number of unique words:  552


In [0]:
def getUniqueChars(text):
  uniqueChars = list(set([char for char in text.lower()]))
  uniqueChars.sort()
  return uniqueChars

In [10]:
print('There are {} unique characters in the dataset, counting lower and upper case letters as one letter'.format(len(getUniqueChars(transcript))))
print(getUniqueChars(transcript))

There are 35 unique characters in the dataset, counting lower and upper case letters as one letter
['\n', ' ', '"', "'", ',', '-', '.', ';', '?', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', '\ufeff']


In [0]:
import string

def cleanupText(text):
  wanted_chars = string.digits + string.ascii_letters + "'" + '".,:;!?-() \n'
  print(wanted_chars)
  return ''.join(x for x in text if x in (wanted_chars))

In [12]:
transcript = cleanupText(transcript)

0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ'".,:;!?-() 



In [13]:
print('After cleanup there are {} unique characters in the dataset'.format(len(getUniqueChars(transcript))))
print(getUniqueChars(transcript))

After cleanup there are 34 unique characters in the dataset
['\n', ' ', '"', "'", ',', '-', '.', ';', '?', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y']


In [0]:
punctuation_dict = {
        '.':'||period||',
        ',':'||comma||',
        '"':'||quote||',
        ';':'||semicolon||',
        '!':'||exclamation||',
        '?':'||question||',
        '(':'||leftPar||',
        ')':'||rightPar||',
        '-':'||dash||',
        '\n':'||return||',
        "'":'||single_quote||'
    }

In [0]:
def tokenizePunctuation(text):
  for key, value in punctuation_dict.items():
    text = text.replace(key, ' {} '.format(value))
  return text

In [0]:
transcript = tokenizePunctuation(transcript)


In [17]:
print(transcript[:200])

It was late and every one had left the cafe except an old man  ||return|| who sat in the shadow the leaves of the tree made against the  ||return|| electric light ||period||  In the day time the stree


In [0]:
unique_words = getUniqueWords(transcript)

In [19]:
print('The number of unique words is: ', len(unique_words))

The number of unique words is:  421


In [0]:
from collections import Counter

def createLookupTables(text, padding_symbol):
  text = text.lower().split()
  text.append(padding_symbol)
  count = Counter(text)
  vocabulary = sorted(count, key=count.get, reverse=True)
  word_to_int = {word:idx for idx, word in enumerate(vocabulary)}
  int_to_word = {idx:word for idx, word in enumerate(vocabulary)}
  return (word_to_int,int_to_word)
  

In [0]:
PADDING_SYMBOL = '<PAD>'

In [0]:
word_to_int,int_to_word = createLookupTables(transcript, PADDING_SYMBOL)

In [23]:
for i in range(5):
  print(int_to_word[i])

||return||
||quote||
||period||
the
he


In [0]:
def changeIntoInts(text):
  return [word_to_int[word] for word in text.lower().split()]

In [0]:
transcript_int = changeIntoInts(transcript)

In [26]:
print(transcript_int[:20])

[9, 11, 76, 6, 131, 77, 78, 196, 3, 39, 132, 55, 18, 19, 0, 32, 79, 24, 3, 93]


In [0]:
import pickle

In [0]:
def save_preprocessed(word_to_int, int_to_word, punctuation_dict, transcript_int, path):  
  with open(path, 'wb') as f:
    pickle.dump((word_to_int, int_to_word, punctuation_dict, transcript_int), f)
    print('Saved preprocessed data in', path)

In [0]:
def load_preprocessed(path):
  return pickle.load(open(path, mode='rb'))

In [0]:
import datetime

project_dir = "/content/drive/My Drive/Text Generator/"

In [31]:
#saving current preprocessed data
path = project_dir + 'preprocessed' + str(datetime.datetime.now()) +'.p'
save_preprocessed(word_to_int, int_to_word, punctuation_dict, transcript_int, path)

Saved preprocessed data in /content/drive/My Drive/Text Generator/preprocessed2019-07-30 17:17:30.349953.p


In [0]:
def save_preprocessed(word_to_int, int_to_word, punctuation_dict, transcript_int, path):  
  with open(path, 'wb') as f:
    pickle.dump((word_to_int, int_to_word, punctuation_dict, transcript_int), f)
    print('Saved preprocessed data in', path)

In [0]:
def load_preprocessed(path):
  return pickle.load(open(path, mode='rb'))

In [0]:
import torch
from torch.utils.data import TensorDataset, DataLoader

def createDataloader(text_int, sequence_length, batch_size):
  
  #first we need to convert the whole dataset into features and targets lists
  features, target = [], []
  
  #we loop through the whole dataset, moving one element at a time
  #we start at the first element and end at the element that is sequence_length from the end of the dataset
  
  for sequence_no in range(len(text_int)-sequence_length):
    features.append(text_int[sequence_no:sequence_no+sequence_length])
    target.append(text_int[sequence_no+sequence_length])
    
  #then we convert the dataset into the Tensor Dataset, which accepts NumPy arrays as parameter
  #Tensor Dataset needs features and target nparrays as parameters
  
  dataset = TensorDataset(torch.from_numpy(np.array(features)), torch.from_numpy(np.array(target)))

  #once we have the appropriate dataset, we can create the dataloder
  
  dataloader = DataLoader(dataset, shuffle=True, batch_size=batch_size)
  
  return dataloader

In [35]:
sample_loader = createDataloader(transcript_int[:50], sequence_length=5, batch_size=10)
sample_iterable = iter(sample_loader)
sample_feature, sample_target = sample_iterable.next()

print('Sample feature batch:')
print(sample_feature)
print('Sample target:')
print(sample_target)

Sample feature batch:
tensor([[  3, 198, 199,   3,  56],
        [  3,  94,  22,   3, 133],
        [  2,  24,   3, 198, 199],
        [197, 134,   3,   0, 135],
        [  3,  39, 132,  55,  18],
        [196,   3,  39, 132,  55],
        [ 18,  19,   0,  32,  79],
        [ 26,   0,  33,   3, 201],
        [132,  55,  18,  19,   0],
        [  6, 131,  77,  78, 196]])
Sample target:
tensor([ 11, 197,   3,  43,  19,  18,  24, 202,  32,   3])


In [0]:
import torch.nn as nn

class ScriptGenModel(nn.Module):
  
  #here we define the structure of the model
  
  def __init__(self, vocab_size, output_size, embedding_dim, hidden_dim, n_layers, dropout=0.5):
    super(ScriptGenModel, self).__init__()
    
    #saving the parameters for future use
    self.output_size = output_size
    self.n_layers = n_layers
    self.hidden_dim = hidden_dim
        
    # defining model layers
    self.embedding = nn.Embedding(vocab_size, embedding_dim)
    self.lstm = nn.LSTM(embedding_dim, hidden_dim, n_layers, dropout=dropout, batch_first=True)
    self.fc = nn.Linear(hidden_dim, output_size)
    
  #here we decide, how the forward pass of the model should be performed
  
  def forward(self, nn_input, hidden):
    
    batch_size = nn_input.size(0)

    embeds = self.embedding(nn_input)
    lstm_out, hidden = self.lstm(embeds, hidden)
    lstm_out = lstm_out.contiguous().view(-1, self.hidden_dim)
    out = self.fc(lstm_out)

    out = out.view(batch_size, -1, self.output_size)
    out = out[:, -1]
    return out, hidden
  
  #here we define the way to initialise the hidden state
  
  def init_hidden(self, batch_size):
    weight = next(self.parameters()).data

    if (train_on_gpu):
      hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().cuda(),
      weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().cuda())
    else:
      hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_(),
      weight.new(self.n_layers, batch_size, self.hidden_dim).zero_())
    return hidden

In [0]:
def forward_back_prop(scriptGenModel, optimizer, criterion, inp, target, hidden):
  # move data to GPU, if available
  if(train_on_gpu):
    inp, target = inp.cuda(), target.cuda()
    
  # perform backpropagation and optimization
  hidden = tuple([each.data for each in hidden])

  scriptGenModel.zero_grad()
    
  output, hidden = scriptGenModel(inp, hidden)

  loss = criterion(output, target)
  loss.backward()
  nn.utils.clip_grad_norm_(scriptGenModel.parameters(), 3)
  optimizer.step()
  return loss.item(), hidden

In [0]:
def train_scriptGenModel(scriptGenModel, batch_size, optimizer, criterion, n_epochs, show_every_n_batches=100):
    batch_losses = []
    
    scriptGenModel.train()

    print("Training for %d epoch(s)..." % n_epochs)
    for epoch_i in range(1, n_epochs + 1):
        
        # initialize hidden state
        hidden = scriptGenModel.init_hidden(batch_size)
        
        for batch_i, (inputs, labels) in enumerate(train_loader, 1):
            
            # make sure you iterate over completely full batches, only
            n_batches = len(train_loader.dataset)//batch_size
            if(batch_i > n_batches):
                break
            
            # forward, back prop
            loss, hidden = forward_back_prop(scriptGenModel, optimizer, criterion, inputs, labels, hidden)          
            # record loss
            batch_losses.append(loss)

            # printing loss stats
            if batch_i % show_every_n_batches == 0:
                print('Epoch: {:>4}/{:<4}  Loss: {}\n'.format(
                    epoch_i, n_epochs, np.average(batch_losses)))
                batch_losses = []

    # returns a trained rnn
    return scriptGenModel

In [0]:
# Parameters for data preprocessing

sequence_length = 8
batch_size = 64

# Training parameters

num_epochs = 10
learning_rate = 0.001

# Model parameters

vocab_size = len(word_to_int)
output_size = vocab_size
embedding_dim = 300
hidden_dim = 512
n_layers = 2

# Show stats for every n number of batches

show_every_n_batches = 500

In [0]:
def save_model(filename, scriptGenModel):
  model_save_name = filename + ".pt"
  path = project_dir + model_save_name
  torch.save(scriptGenModel, path)

In [0]:
def load_model(filename):
    path = project_dir + filename + ".pt"
    return torch.load(path)

In [42]:
#checking if GPU is available

train_on_gpu = torch.cuda.is_available()
if not train_on_gpu:
  print('There is no GPU available. Please consider switching to GPU for training the model.')
else:
  print("GPU available. You're good to go!") 

GPU available. You're good to go!


In [0]:
train_loader = createDataloader(transcript_int, sequence_length, batch_size)

In [0]:
# creaing the model
scriptGenModel = ScriptGenModel(vocab_size, output_size, embedding_dim, hidden_dim, n_layers, dropout=0.5)

#deciding on optimizer and loss functions
optimizer = torch.optim.Adam(scriptGenModel.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss()

In [45]:
print(scriptGenModel)

ScriptGenModel(
  (embedding): Embedding(422, 300)
  (lstm): LSTM(300, 512, num_layers=2, batch_first=True, dropout=0.5)
  (fc): Linear(in_features=512, out_features=422, bias=True)
)


In [46]:
#moving the model to GPU if it's available
if train_on_gpu:
    scriptGenModel.cuda()

#running the trainining loop
trained_scriptGenModel = train_scriptGenModel(scriptGenModel, batch_size, optimizer, criterion, num_epochs, show_every_n_batches)

#Setup for saving the trained model to Google Drive
current_time = datetime.datetime.now() 
model_name = 'trained_scriptGenModel' + str(current_time)
save_model(model_name, trained_scriptGenModel)

Training for 10 epoch(s)...


/usr/local/lib/python3.6/dist-packages/torch/serialization.py:256: UserWarning: Couldn't retrieve source code for container of type ScriptGenModel. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


In [0]:
import torch.nn.functional as F

def generate(scriptGenModel, prime_id, int_to_vocab, punctuation_dict, pad_value, predict_len=100):

    scriptGenModel.eval()
    
    # create a sequence (batch_size=1) with the prime_id
    current_seq = np.full((1, sequence_length), pad_value)
    current_seq[-1][-1] = prime_id
    predicted = [int_to_vocab[prime_id]]
    
    for _ in range(predict_len):
        if train_on_gpu:
            current_seq = torch.LongTensor(current_seq).cuda()
        else:
            current_seq = torch.LongTensor(current_seq)
        
        # initialize the hidden state
        hidden = scriptGenModel.init_hidden(current_seq.size(0))
        
        # get the output of the model
        
        ##### Added next 2 lines to fix numpy bug  ####
        scriptGenModel.cpu()
        current_seq = current_seq.cpu()
        hidden = hidden[0].cpu(), hidden[1].cpu()
        #print(" curs:", current_seq.device, "hidden[0]", hidden[0].device, "hidden[1]", hidden[1].device)
        output, _ = scriptGenModel(current_seq, hidden)
        
        # get the next word probabilities
        p = F.softmax(output, dim=1).data
        if(train_on_gpu):
            p = p.cpu() # move to cpu
         
        # use top_k sampling to get the index of the next word
        top_k = 5
        p, top_i = p.topk(top_k)
        top_i = top_i.numpy().squeeze()
        
        # select the likely next word index with some element of randomness
        p = p.numpy().squeeze()
        word_i = np.random.choice(top_i, p=p/p.sum())
        
        # retrieve that word from the dictionary
        word = int_to_vocab[word_i]
        predicted.append(word)     
        
        # the generated word becomes the next "current sequence" and the cycle can continue
        current_seq = np.roll(current_seq, -1, 1)
        current_seq[-1][-1] = word_i
    
    gen_sentences = ' '.join(predicted)
    
    # Replace punctuation tokens
    for key, token in punctuation_dict.items():
        ending = ' ' if key in ['\n', '(', '"'] else ''
        gen_sentences = gen_sentences.replace(' ' + token.lower(), key)
    gen_sentences = gen_sentences.replace('\n ', '\n')
    gen_sentences = gen_sentences.replace('( ', '(')
    
    # return all the sentences
    return gen_sentences

In [48]:
#we check if the parameters for text generation are loaded, and if not we load them in

try:
  if (word_to_int and int_to_word and punctuation_dict and PADDING_SYMBOL):
    print('Data for text generation present')
except:
  path = project_dir+'preprocessed.p'
  word_to_int, int_to_word, punctuation_dict, transcript_int = load_preprocessed(path)

Data for text generation present


In [49]:
gen_length = 500 
prime_word = 'What'

generated_script = generate(trained_scriptGenModel, word_to_int[prime_word.lower()], int_to_word, punctuation_dict, word_to_int[PADDING_SYMBOL], gen_length)
print(generated_script)

what."


" what do you want?"


" no. you have everything."


" no," the waiter said. the waiter went away.


" no," the waiter said. the old man looked at the street. the street
light shone on the number on the street where the street,
leaving half to a peseta. he


the old man looked at him." i know."


" no, hombre," the waiter said. the waiter went over him him down."


" what should you want to look him."


" i should have killed yourself last i week." he said.







" no."


" i want to go home to bed. i am never get into before o o' clock. he kind of a wife waiting in bed
for me."


" i'' m not." the waiter
who. the waiter took the bottle inside the street and looked at the table
with a wife pressure coffee too.


" no. finished."


" no, hombre, he would go at to his colleague."


" what about?"


" you have everything."


" you have everything i have."


" he' m drunk every night," said the younger and went went over. the old man motioned with his wife,."


" i want to go to 

In [50]:
gen_length = 500 
prime_word = 'Man'

generated_script = generate(trained_scriptGenModel, word_to_int[prime_word.lower()], int_to_word, punctuation_dict, word_to_int[PADDING_SYMBOL], gen_length)
print(generated_script)

man."


" no," said the waiter.


" he' s plenty. i am all a very and a clean and pleasant
out. the guard will went up him."


" what do you do?"


" he had a wife once too."


" he' s drunk every him."


" you want' t you want to stay that at him." you have everything."


" i' t want to look at him."


" you should have killed last last last week, he he would deaf home. he













" no, hombre," he said. the old man looked at
him. the waiter went over to him.


" no," said the old man.


" no. i have never confidence. i never get to bed before three
o' s. he would lie the bed and finally to him down
" the waiter wiped the edge of the tree of the tree and made in the




" no," the waiter said. he was only in a hurry.


" what about?"


the barman watched him go him."


" no, you, you do not want mu-
sic. nor can you stand before a bar with dignity although that

is are that is provided for these hours. he might be better with a wife."


" no. i am reluctant to up up because becau